In [83]:
#import the files 
import re 
import nltk
import ssl
import pandas as pd 
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB


In [ ]:

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [2]:
stop_words = list(set(stopwords.words('english')))
# JJ/R/S = adjective/comparitive/superlative
allowed_word_types = ["JJ","JJR","JJS"]

In [3]:
#open the data 
test_data = pd.read_csv('testing_set.csv')
train_data = pd.read_csv('training_set.csv')

train_data.head()

,Unnamed: 0,Review,Liked
0,300,Good beer & drink selection and good food sele...,1
1,301,Please stay away from the shrimp stir fried no...,0
2,302,The potato chip order was sad... I could proba...,0
3,303,Food was really boring.,0
4,304,Good Service-check!,1


In [4]:
#input tokenized list of words from review 
#return tokenized list without stop words 
def remove_stopword(data_input):
    cleaned_words = []
    for word in data_input: 
        if word not in stop_words: 
            cleaned_words.append(word)
    return (cleaned_words)

            
    

In [5]:
# input tokenized and pos tagged list 
# return list of just adjectives
def only_adjectives(data_input):
    cleaned_words = []
    for word, pos in data_input: 
        if pos in allowed_word_types:
            cleaned_words.append(word)
    return (cleaned_words)

In [6]:
#clean the training set for everything except for adjectives 
corpus = []

#lower case 
train_data["Review"] = train_data["Review"].apply(lambda x:x.lower())
#get rid of punctuation
train_data["Review"] = train_data["Review"].apply(lambda x:re.sub('[^a-zA-Z]',' ',x))
#split
train_data["Review"] = train_data["Review"].apply(lambda x:x.split())
#remove the stop words 
train_data["Review"] = train_data["Review"].apply(lambda x: remove_stopword(x))

#pos tag --> creates tuple (word,pos-tag)
train_data["Review"] = train_data["Review"] = train_data["Review"].apply(lambda x:nltk.pos_tag(x))
#Select only adjectives and remove pos-tags
train_data["Review"] = train_data["Review"].apply(lambda x:only_adjectives(x))

#maybe removing all empty is good or can tell about sentiment 



In [7]:
train_data.head(10)

,Unnamed: 0,Review,Liked
0,300,"[good, good]",1
1,301,[shrimp],0
2,302,[many],0
3,303,[boring],0
4,304,[good],1
5,305,[],0
6,306,[],0
7,307,"[much, atrocious]",0
8,308,[outdoor],1
9,309,[good],1


In [8]:

#create list of all adjectives from reviews in train_data
all_review_train = []
for row in train_data["Review"]:
    for word in row:
        all_review_train.append(word)

In [9]:
# BoW using count vectorizer 
cv = CountVectorizer(max_features = 1500)
cv.fit(all_review_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=1500, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [133]:
#cv.vocabulary_

In [20]:
train_data_2 = pd.read_csv('training_set.csv')


In [21]:
#returns a vector of a input sentence based on vocabulary 
def vectorize_sentence(sentence):
    text = [sentence]
    return (cv.transform(text).toarray())

In [22]:
train_data_2.head()

,Unnamed: 0,Review,Liked
0,300,Good beer & drink selection and good food sele...,1
1,301,Please stay away from the shrimp stir fried no...,0
2,302,The potato chip order was sad... I could proba...,0
3,303,Food was really boring.,0
4,304,Good Service-check!,1


In [23]:
# Vectorize the reviews in training dataset based on vocab in cv 
train_data_2["Review"] = train_data_2["Review"].apply(lambda x:vectorize_sentence(x))

In [73]:
X_train = train_data_2["Review"].values
y_train = train_data_2["Liked"].values

In [76]:
X_train.shape

(700,)

In [81]:
len(X_train[0][0])

363

In [82]:
X_train_2D = np.reshape(X_train,)

IndexError: list index out of range

In [72]:
X_train.shape

AttributeError: 'list' object has no attribute 'shape'

In [57]:
classifier = BernoulliNB(alpha = .8)
classifier.fit(X_train, y_train)

ValueError: Found array with dim 3. Estimator expected <= 2.

In [64]:
#Import Library
from sklearn import svm
import numpy as np

X=np.array([
[[25,25,25],[0,0,0],[0,0,0]],
[[25,0,0],[25,0,0],[25,0,0]],
[[75,75,75],[75,75,75],[75,75,75]]
       ])
X = X.reshape(X.shape[0], -1)
y=np.array([-1,1,1])

C=10

model = svm.SVC(kernel='rbf', C=10, gamma=0.6) 


model.fit(X, y)
model.score(X, y)

1.0

In [66]:
print (type(X))

<class 'numpy.ndarray'>


In [69]:
X.shape

(3, 9)

In [70]:
X


array([[25, 25, 25,  0,  0,  0,  0,  0,  0],
       [25,  0,  0, 25,  0,  0, 25,  0,  0],
       [75, 75, 75, 75, 75, 75, 75, 75, 75]])